# AIPI531: Homework 3


## Install required packages

In [1]:
! python3 -m pip install trfl==1.2.0

In [2]:
import pandas as pd

## Mount file system

In [3]:
# Mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# change current directory to the project folder
# The original source code could be found from:
# https://drive.google.com/file/d/185KB520pBLgwmiuEe7JO78kUwUL_F45t/
PROJ_DIR = '/content/drive/My Drive/Courses/AIPI531/HW3/SA2C_code/Kaggle'
%cd $PROJ_DIR
! ls

/content/drive/My Drive/Courses/AIPI531/HW3/SA2C_code/Kaggle
data		      __pycache__		   SASRecModules.py	    split_data.py
DQN_NS.py	      replay_buffer.py		   SNQN_AddFeatures_new.py  test.py
NextItNetModules.py   report_SNQN_AddFeatures.txt  SNQN_AddFeatures.py	    upgradeLog.txt
pop.py		      report_SNQN.txt		   SNQN_new.py		    utility.py
preprocess_kaggle.py  SA2C.py			   SNQN.py


## Upgrade Tensorflow version 1-->2

Need to manually modify the original source code

1.   Disable eager execution by adding the following code on the first line of the init function `tf.compat.v1.disable_eager_execution()`
2.   Replace `tf.contrib.layers.fully_connected` with `tf.compat.v1.layers.dense`





In [ ]:
!tf_upgrade_v2 \
  --infile 'SNQN.py' \
  --outfile 'SNQN_new.py' \
  --reportfile report_SNQN.txt

2023-12-12 06:25:30.729300: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 06:25:30.729360: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 06:25:30.729407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 06:25:30.737609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 06:25:31.951386: W tensorflow/compiler/

In [ ]:
!tf_upgrade_v2 \
  --infile 'SNQN_AddFeatures.py' \
  --outfile 'SNQN_AddFeatures_new.py' \
  --reportfile report_SNQN_AddFeatures.txt

2023-12-12 06:25:35.011422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 06:25:35.011477: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 06:25:35.011515: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 06:25:35.022861: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 06:25:36.695985: W tensorflow/compiler/

## Without item features

In [ ]:
! python SNQN_new.py --model=GRU --epoch=6

2023-12-11 20:09:14.404912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 20:09:14.404961: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 20:09:14.404991: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 20:09:14.412946: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 20:09:15.629118: W tensorflow/c

## With features

Need to download the full dataset from:
https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset

Put the dataset under the folder `AIPI531\HW3\SA2C_code\Kaggle\data`

### Need to preprocess data to genrerate features

In [5]:
def load_dataframes(sorted_events, n_files=2, path_name="./item_properties_part"):
    """
    Load CSV data into pandas DataFrames.
    """
    # Load sorted events data from a CSV file into a DataFrame
    sorted_events_df = pd.read_csv(sorted_events)
    # Load item properties data from multiple CSV files into a list of DataFrames
    dfs = [pd.read_csv(f"{path_name}{i + 1}.csv") for i in range(n_files)]
    # Concatenate all item properties DataFrames into a single DataFrame
    item_features_df = pd.concat(dfs, ignore_index=True)
    return sorted_events_df, item_features_df

def preprocess_item_features(sorted_events_df, item_features_df):
    """
    Preprocess item features DataFrame by filtering and cleaning data.
    """
    # Get unique item IDs from sorted events DataFrame
    unique_item_ids = sorted_events_df["item_id"].unique()
    # Filter item features DataFrame for items present in sorted events
    item_features_df = item_features_df[item_features_df["itemid"].isin(unique_item_ids)].drop_duplicates()
    # Drop rows where item is marked as unavailable
    item_features_df = item_features_df[~((item_features_df["property"] == "available") & (item_features_df["value"] == '0'))]
    # Create a new column combining property and value as a single string
    item_features_df["property_value"] = item_features_df["property"].str.strip() + item_features_df["value"].str.strip()
    # Drop timestamp column and any duplicate rows
    return item_features_df.drop(["timestamp"], axis=1).drop_duplicates()

def one_hot_encode_features(sorted_events_df, item_features_df, top_features=500):
    """
    One hot encode the top item features.
    """
    # Get unique sorted item IDs and sort them
    unique_event_items = sorted_events_df["item_id"].unique()
    unique_event_items.sort()
    # Get the top features based on occurrence
    properties = item_features_df["property_value"].value_counts().head(top_features).index.tolist()
    one_hot_encoded = []
    itemids = []

    # Iterate over each item and create a one-hot encoded row for its features
    for _, item in enumerate(unique_event_items):
        item_properties = set(item_features_df[item_features_df["itemid"] == item]["property_value"].unique())
        encoded_row = [1 if prop in item_properties else 0 for prop in properties]
        one_hot_encoded.append(encoded_row)
        itemids.append(item)

    print("One hot encoding done.")
    return pd.DataFrame(one_hot_encoded), itemids

def create_feature_matrix(sorted_events, n_files, path_name, top_features=500):
    """
    Create and return a matrix of one hot encoded item features.
    """
    # Load and preprocess data
    sorted_events_df, item_features_df = load_dataframes(sorted_events, n_files, path_name)
    item_features_df = preprocess_item_features(sorted_events_df, item_features_df)

    # Perform one hot encoding of features
    return one_hot_encode_features(sorted_events_df, item_features_df, top_features)


In [ ]:
# gernerate item features
df_item_feature, _ = create_feature_matrix(sorted_events='./data/sorted_events.csv', n_files=2,
                                           path_name="./data/item_properties_part", top_features=500)
df_item_feature.to_csv('./data/item_features.csv', index=False)

One hot encoding done.


In [ ]:
! python SNQN_AddFeatures_new.py --model=GRU --epoch=6 --mix_lambda=0.1

2023-12-12 02:17:12.913681: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 02:17:12.913742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 02:17:12.913801: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 02:17:12.925654: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 02:17:14.633193: W tensorflow/c

In [6]:
! python SNQN_AddFeatures_new.py --model=GRU --epoch=6 --mix_lambda=0.2

2023-12-12 07:23:19.965076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 07:23:19.965130: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 07:23:19.965175: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 07:23:19.972512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 07:23:21.016765: W tensorflow/compiler/

In [5]:
! python SNQN_AddFeatures_new.py --model=GRU --epoch=6 --mix_lambda=0.5

2023-12-12 13:58:45.547037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 13:58:45.547089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 13:58:45.547127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 13:58:45.554651: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 13:58:46.595736: W tensorflow/compiler/